In [42]:
from keras.models import Sequential
"""Import from keras_preprocessing not from keras.preprocessing, because Keras may or maynot contain the features discussed here depending upon when you read this article, until the keras_preprocessed library is updated in Keras use the github version."""
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
#from tensorflow.keras import optimizers #., optimizers
import pandas as pd
import numpy as np

import tensorflow as tf


#Importing all the relevant library
%matplotlib inline
import h5py, os
#from functions import transforms as T
#from functions.subsample import MaskFunc
from scipy.io import loadmat
from torch.utils.data import DataLoader
import numpy as np
import torch
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import glob
#from functions import transforms as T 
#from functions.subsample import MaskFunc
from PIL import Image

In [43]:
def load_data_path(train_data_path, val_data_path, test_data_path):
    """ Go through each subset (training, validation) and list all 
    the file names, the file paths and the slices of subjects in the training and validation sets 
    """

    data_list = {}
    train_val_test = ['train', 'val', 'test']
    data_path = [train_data_path, val_data_path, test_data_path]
      
    for i in range(len(data_path)):

        data_list[train_val_test[i]] = []
        
        which_data_path = data_path[i]
        tr = 0
        te = 0
        alfa = 0
        for fname in sorted(os.listdir(which_data_path + '/images')):
            if fname != ".DS_Store":

            
                subject_data_path = os.path.join(which_data_path + '/images', fname)
                     
                if not os.path.isfile(subject_data_path): continue 
            
          #  im_frame = Image.open(subject_data_path)

            #get information from text file
            # this will return a tuple of root and extension
                split_tup = os.path.splitext(fname)

  
            # extract the file name and extension
                file_name = split_tup[0]
                file_path = os.path.join(which_data_path + '/texts', file_name) + '.txt'
                f = open(os.path.join(which_data_path + '/texts', file_name) + '.txt', 'r')
                line = f.readlines()[1]
            
                fields = line.split(',')
                tr = int(fields[0])
                te = int(fields[1])
                alfa = int(fields[2])
                f.close()
                
            # the first 5 slices are mostly noise so it is better to exlude them
                data_list[train_val_test[i]] += [(fname, tr)]
    
    return data_list

In [44]:
data_list = load_data_path ("data/train", "data/val", "data/test")

In [45]:
train_data = data_list['train']
val_data = data_list['val']
test_data = data_list['val']

In [46]:
train_df = pd.DataFrame(train_data, columns=['fnames', 'labels'])
train_df['labels']= train_df['labels'].astype(str)
train_df['fnames']= train_df['fnames'].astype(str)
val_df = pd.DataFrame(val_data, columns=['fnames', 'labels'])
val_df['labels']= val_df['labels'].astype(str)
val_df['fnames']= val_df['fnames'].astype(str)
test_df = pd.DataFrame(test_data, columns=['fnames', 'labels'])
test_df['labels']= test_df['labels'].astype(str)
test_df['fnames']= test_df['fnames'].astype(str)
labels = train_df.labels.unique()

In [47]:
num_class = len(labels)
num_class

20

In [48]:
train_df.describe

<bound method NDFrame.describe of         fnames labels
0        1.png      5
1       10.png     12
2      100.png     11
3     1000.png     13
4     1001.png     18
...        ...    ...
2995   995.png     19
2996   996.png     14
2997   997.png      9
2998   998.png      3
2999   999.png     16

[3000 rows x 2 columns]>

In [49]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(
      dataframe=train_df,
      directory="./data/train/images/",
      x_col="fnames",
      y_col="labels",
      batch_size=32,
      seed=42,
      shuffle=True,
      class_mode="categorical",
      target_size=(224,224))



valid_generator=test_datagen.flow_from_dataframe(
      dataframe=val_df,
      directory="./data/val/images/",
      x_col="fnames",
      y_col="labels",
      batch_size=32,
      seed=42,
      shuffle=True,
      class_mode="categorical",
     target_size=(224,224))


Found 3000 validated image filenames belonging to 20 classes.
Found 500 validated image filenames belonging to 20 classes.


In [50]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_df,
      directory="./data/test/images",
      x_col="fnames",
      batch_size=1,
      seed=42,
      shuffle=False,
      class_mode=None,
      target_size=(224,224))

Found 400 validated image filenames.


/Users/hoangvo/opt/anaconda3/envs/coursework/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 100 invalid image filename(s) in x_col="fnames". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [51]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_class, activation='sigmoid'))
model.compile(tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

In [52]:
n_epochs = 100 #10

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=n_epochs
)

/Users/hoangvo/opt/anaconda3/envs/coursework/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
93/93 [==============================] - 289s 3s/step - loss: 0.3090 - accuracy: 0.0443 - val_loss: 0.2069 - val_accuracy: 0.0417
Epoch 2/100
93/93 [==============================] - 282s 3s/step - loss: 0.2162 - accuracy: 0.0442 - val_loss: 0.2158 - val_accuracy: 0.0583
Epoch 3/100
93/93 [==============================] - 278s 3s/step - loss: 0.2133 - accuracy: 0.0486 - val_loss: 0.2126 - val_accuracy: 0.0500
Epoch 4/100
93/93 [==============================] - 308s 3s/step - loss: 0.2096 - accuracy: 0.0578 - val_loss: 0.2063 - val_accuracy: 0.0479
Epoch 5/100
93/93 [==============================] - 307s 3s/step - loss: 0.2086 - accuracy: 0.0539 - val_loss: 0.2123 - val_accuracy: 0.0521
Epoch 6/100
48/93 [==============>...............] - ETA: 2:25 - loss: 0.2075 - accuracy: 0.0612

In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

In [ ]:
test_generator

In [ ]:
pred_bool = (pred >0.5)

In [ ]:
predictions=[]
labels = train_generator.class_indices
labels = dict((v,k) for k,v in labels.items())
for row in pred_bool:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(label[index])
    predictions.append(",".join(l))
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
#results.to_csv("results.csv",index=False)

In [ ]:
results

In [ ]:
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

#load the image
my_image = load_img('data/test/images/1.png', target_size=(224, 224))

#preprocess the image
my_image = img_to_array(my_image)
my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
my_image = preprocess_input(my_image)

#make the prediction
prediction = model.predict(my_image)

In [ ]:
# save the model
filename2 = 'model_tr.h5' % (step+1)
model.save(filename2)

In [ ]:
from keras.models import load_model
# load model
model = load_model('model_tr.h5')